In [60]:
def intervalo(modelo, results):
    mean = results[0]
    dv = results[1]
    print("-"*10,modelo,"-"*10)
    print('Acurácia média: {}'.format(mean))
    print('Intervalo de acurácia: [{} ~ {}]\n'
           .format((mean - 2*dv), (mean + 2*dv)))
    

In [66]:
import pandas as pd

def ApplyesKFold(x_axis, y_axis, k):
    # Modelo Linear.
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import ElasticNet
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.tree import DecisionTreeClassifier


    # Cross-Validation modelos.
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold

    # Configuração KFold.
    # shuffle=True, Shuffle (embaralhar) o dataset.
    kfold = KFold(n_splits=k, shuffle=True)

    # Axis
    x = x_axis
    y = y_axis

    # Instancias de modelos.
    linearRegression = LinearRegression()
    elasticNet = ElasticNet()
    ridge = Ridge()
    lasso = Lasso()
    decisionTree = DecisionTreeClassifier(criterion="gini")


    # Aplica o KFold aos modelos.
    linearRegression_result = cross_val_score(linearRegression, x, y, cv=kfold)
    elasticNet_result = cross_val_score(elasticNet, x, y, cv=kfold)
    ridge_result = cross_val_score(ridge, x, y, cv=kfold)
    lasso_result = cross_val_score(lasso, x, y, cv=kfold)
    decisionTree_result = cross_val_score(decisionTree, x, y, cv=kfold)

    # Cria um dicionário para armazenar modelos lineares.
    dic_models = {
        "LinearRegression": [linearRegression_result.mean(),linearRegression_result.std()],
        "ElasticNet": [elasticNet_result.mean(),elasticNet_result.std()],
        "Ridge": [ridge_result.mean(),ridge_result.std()],
        "Lasso": [lasso_result.mean(),lasso_result.std()],
        "DecisionTree": [decisionTree_result.mean(), decisionTree_result.std()]
    }
    # Selecione o melhor modelo.
    # print(dic_models.)
   
    for modelo, result in dic_models.items():
        intervalo(modelo, result)

    bestModel = max(dic_models, key=dic_models.get)

    # print("Media Linear Regression: {0}\n"
    #       "Media Elastic Net: {1}\n"
    #       "Media Ridge: {2}\n"
    #       "Media Lasso: {3}\n"
    #       "Media Decision Tree: {4}".format(
    #           linearRegression_result.mean(), 
    #           elasticNet_result.mean(), 
    #           ridge_result.mean(), 
    #           lasso_result.mean(), 
    #           decisionTree_result.mean()
    #           ))
    # print()
    print("O melhor modelo e: {0} com valor: {1}".format(
        bestModel, dic_models[bestModel][0]))


In [67]:
# df = pd.read_csv("Admission_Predict.csv")
df = pd.read_csv("datasetMobile/train.csv")
# df.drop('Serial No.', axis=1, inplace=True)

x = df.drop('price_range', axis=1)
y = df['price_range']

ApplyesKFold(x, y, 10)

---------- LinearRegression ----------
Acurácia média: 0.9163458985367814
Intervalo de acurácia: [0.902808145371363 ~ 0.9298836517021998]

---------- ElasticNet ----------
Acurácia média: 0.916809583703133
Intervalo de acurácia: [0.9092290050427382 ~ 0.9243901623635279]

---------- Ridge ----------
Acurácia média: 0.9160908601864746
Intervalo de acurácia: [0.902492559448957 ~ 0.9296891609239921]

---------- Lasso ----------
Acurácia média: 0.9164753570345038
Intervalo de acurácia: [0.9096618429170932 ~ 0.9232888711519144]

---------- DecisionTree ----------
Acurácia média: 0.8290000000000001
Intervalo de acurácia: [0.778245689838202 ~ 0.8797543101617982]

O melhor modelo e: ElasticNet com valor: 0.916809583703133
